<a href="https://colab.research.google.com/github/F1ame21/Primat_Labs/blob/main/lab4/Jacobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from numpy import random
import math
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy import stats
from scipy.sparse import random

In [3]:
def empty_matrix(n, m, format):
    """Пустой нулевой двумерный массив
    :param n: количество строк
    :param m: количество столбцов
    :return: python array
    """
    Matrix = scipy.sparse.__dict__[format + "_matrix"]
    return Matrix((n, m))

In [4]:
def generate_big_matrix(n, p, format):
    """Генерирует большую матрицу nxn разреженности p
    :param n: размерность или порядок матрицы
    :param p: отношение ненулевых клеток к nxn
    :return: матрица A в разреженном виде
    """
    rvs = lambda s: np.random.uniform(-4, 0, size=s)
    return random(n,n , density=p, format = format, dtype = int, data_rvs = rvs)

In [5]:
def generate_symmetrix_matrix(n, p, format):
  A = generate_big_matrix(n, p, format)
  A = A + A.T - scipy.sparse.diags(A.diagonal())
  return A

In [134]:
def generate_diagonal_domination_matrix(a, k):
  """
  Генерирует тестовое уравнение для решения
  :param a: квадратная матрица коэффициентов a_ij (см. лабу, а также отчет)
  :param k: номер уравнения в последовательности
  :return: пара (A_k, F_k) для уравнения A_k * x_k = F_k
  None - если уравнение несовместно
    """
  n = a.shape[0]
  A_k = empty_matrix(n, n, "lil")
  for i in range(n):
    s1 = -sum(a[i, q] for q in range(0, i))
    s2 = -sum(a[i, q] for q in range(i+1, n))
    s = s1 + s2
    for j in range(n):
      if i == j:
        A_k[i, j] = -s - pow(10, k)
      else:
        A_k[i, j] = a[i, j]
  A_k = A_k.tocsr()
  return A_k

In [7]:
def generate_hilbert_matrix(k):
    """Генерирует тестовое уравнение для решения с матрицей Гильберта
    :param k: номер уравнения в последовательности
    :return: пара (A_k, F_k) для уравнения A_k * x_k = F_k
    None - если уравнение несовместно
    """
    A_k = empty_matrix(k, k, "lil")
    for i in range(k):
        for j in range(k):
            A_k[i, j] = 1.0 / (i + j + 1.0)

    return A_k.tocsr()

In [8]:
def max_element(a):
  n = a.shape[0]
  l = 0; r = 1
  max = a[0, 1]
  for i in range(0, n):
    for j in range(i+1, n):
        if abs(a[i, j]) > abs(max):
          max = a[i, j]
          l = i; r = j
  return max, l, r

In [9]:
def search_alpha(a, i, j):
  arctg = math.atan((2. * a[i, j])/(a[i, i] - a[j, j]))
  return 1./2 * arctg 

In [111]:
def isCorrectAway(max):
  if abs(max) < 1e-7:
    return True
  return False

In [107]:
def main(A, n):
  R_all = 0
  k = 0
  while True:
    max, i, j = max_element(A)
    if isCorrectAway(max):
      return A, R_all, k
    alpha = search_alpha(A, i, j)
    R = np.eye(n)
    R[i, i] = np.cos(alpha)
    R[j, j] = np.cos(alpha)
    R[i, j] = -np.sin(alpha)
    R[j, i] = np.sin(alpha)
    if k != 0:
      R_all = np.dot(R_all, R)
    else: 
      R_all = R
    R_inv = np.transpose(R)
    B = np.dot(R_inv, A)
    A = np.dot(B, R)
    k += 1

In [146]:
def error(a, s, v):
  F = np.dot(V, S)
  B = np.dot(F, np.linalg.matrix_power(V, -1))
  error = abs(a) - abs(B)
  return print(
      f'\n\nМатрицы ошибок:\n{np.round(error, 16)}'  
  ) 

In [149]:
if __name__ == '__main__':
  # A = np.array([[5, 1, 2], [1, 4, 1], [2, 1, 3]])
  #A1 = generate_symmetrix_matrix(5, 1, 'lil')
  # A = generate_diagonal_domination_matrix(A1, 8).toarray()
  A = generate_hilbert_matrix(6).toarray()
  n = A.shape[0]
  print(f'Матрица A:\n{A}\n')
  S, V, k = main(A, n)
  print(
          f'Собственные значения:A:\n{np.diag(np.round(S, 2))}\n\n'
          f'Собственные вектора:\n{np.round(V, 3)}\n\n'
          f'Ходов:\n{k}'
          )
  error(A, S, V)


Матрица A:
[[1.         0.5        0.33333333 0.25       0.2        0.16666667]
 [0.5        0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.33333333 0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.25       0.2        0.16666667 0.14285714 0.125      0.11111111]
 [0.2        0.16666667 0.14285714 0.125      0.11111111 0.1       ]
 [0.16666667 0.14285714 0.125      0.11111111 0.1        0.09090909]]

Собственные значения:A:
[1.62 0.24 0.   0.   0.   0.02]

Собственные вектора:
[[ 0.749 -0.615  0.062 -0.011  0.001  0.24 ]
 [ 0.441  0.211 -0.491  0.18  -0.036 -0.698]
 [ 0.321  0.366  0.535 -0.604  0.241 -0.231]
 [ 0.254  0.395  0.417  0.444 -0.625  0.133]
 [ 0.212  0.388 -0.047  0.441  0.69   0.363]
 [ 0.181  0.371 -0.541 -0.459 -0.272  0.503]]

Ходов:
41


Матрицы ошибок:
[[ 1.e-16  1.e-16 -2.e-16  1.e-16  1.e-16  1.e-16]
 [-1.e-16  1.e-16 -1.e-16  1.e-16  1.e-16  1.e-16]
 [ 1.e-16  1.e-16  0.e+00  1.e-16  0.e+00  1.e-16]
 [-1.e-16  1.e-16  0.e+00  1.e-16  0.e+00 